In [ ]:
import piplite


for package in [
    "wrapt",
    "typing_extensions",
    "mypy_extensions",
    "typing_inspect",
    "pydantic",
    "pandera",
]:
    await piplite.install(package, deps=False)

In [ ]:
import pandas as pd
import pandera as pa
from pandera.typing import DataFrame, Series


class Schema(pa.SchemaModel):
    item: Series[str] = pa.Field(isin=["apple", "orange"], coerce=True)
    price: Series[float] = pa.Field(gt=0)


@pa.check_types(lazy=True)
def transform_data(data: DataFrame[Schema]):
    ...


data = pd.DataFrame.from_records([
    {"item": "applee", "price": 0.5},
    {"item": "orange", "price": -1000}
])


try:
    transform_data(data)
except pa.errors.SchemaErrors as exc:
    display(exc.failure_cases)